In [2]:
import pandas as pd

In [3]:
train = pd.read_csv("../data/liver-data-processed/train.csv")
dev = pd.read_csv("../data/liver-data-processed/devSet.csv")
test = pd.read_csv("../data/liver-data-processed/test.csv")

data = pd.concat([train, dev, test])

data.REGION = data.REGION.astype(int)
cron = data.groupby("REGION")["DAYSWAIT_CHRON"].mean().to_frame().reset_index()
cron.rename(columns = {"DAYSWAIT_CHRON": "WaitingTime"}, inplace = True)
print cron
data = data.loc[:, ("REGION", "GSTATUS", "GTIME")]
N = data.shape[0]

    REGION  WaitingTime
0        1   257.038528
1        2   207.228608
2        3   121.850287
3        4   248.836022
4        5   299.653453
5        6   221.588235
6        7   210.181089
7        8   220.137522
8        9   257.451602
9       10   162.399387
10      11   171.378999


In [4]:
mask_3_m = (data["GTIME"] <= 90) & (data["GSTATUS"] == 1)
mask_6_m = (data["GTIME"] <= 6 * 30) & (data["GSTATUS"] == 1)
mask_1_y = (data["GTIME"] <= 365) & (data["GSTATUS"] == 1)
mask_3_y = (data["GTIME"] <= 3 * 365) & (data["GSTATUS"] == 1)

In [5]:
three_months = data.loc[mask_3_m, ["REGION", "GSTATUS"]].groupby("REGION").count().reset_index()
three_months["GSTATUS"] = 100 - (three_months["GSTATUS"] * 100.0 / N)
three_months.rename(columns = {"GSTATUS": "status_3_m"}, inplace = True)
three_months

,REGION,status_3_m
0,1,99.698304
1,2,98.995900
2,3,98.719830
3,4,99.398938
4,5,98.981922
5,6,99.831097
6,7,99.287112
7,8,99.521247
8,9,99.230034
9,10,99.281288


In [6]:
six_months = data.loc[mask_6_m, ["REGION", "GSTATUS"]].groupby("REGION").count().reset_index()
six_months["GSTATUS"] = 100 - (six_months["GSTATUS"] * 100.0 / N)
six_months.rename(columns = {"GSTATUS": "status_6_m"}, inplace = True)
six_months

,REGION,status_6_m
0,1,99.596962
1,2,98.631302
2,3,98.335430
3,4,99.203243
4,5,98.670907
5,6,99.751887
6,7,99.068120
7,8,99.375641
8,9,99.044823
9,10,99.092582


In [7]:
one_year = data.loc[mask_1_y, ["REGION", "GSTATUS"]].groupby("REGION").count().reset_index()
one_year["GSTATUS"] = 100 - (one_year["GSTATUS"] * 100.0 / N)
one_year.rename(columns = {"GSTATUS": "status_1_y"}, inplace = True)
one_year

,REGION,status_1_y
0,1,99.479312
1,2,98.128087
2,3,97.746016
3,4,98.949306
4,5,98.210791
5,6,99.687820
6,7,98.762930
7,8,99.162473
8,9,98.757106
9,10,98.783897


In [8]:
three_years = data.loc[mask_3_y, ["REGION", "GSTATUS"]].groupby("REGION").count().reset_index()
three_years["GSTATUS"] = 100 - (three_years["GSTATUS"] * 100.0 / N)
three_years.rename(columns = {"GSTATUS": "status_3_y"}, inplace = True)
three_years

,REGION,status_3_y
0,1,99.283618
1,2,97.169416
2,3,96.577672
3,4,98.300485
4,5,97.342978
5,6,99.516587
6,7,98.154878
7,8,98.739633
8,9,98.160703
9,10,98.164197


In [9]:
regions = pd.read_csv("../data/wage_gap_all.csv")
regions = regions.loc[:, ["Region", "State"]]
regions.columns = ["REGION", "STATE"]
print regions.loc[:10, :]

    REGION STATE
0        1    RI
1        1    MA
2        1    CT
3        1    ME
4        1    NH
5        2    DE
6        2    DC
7        2    MD
8        2    NJ
9        2    PA
10       2    WV


In [10]:
merged_data = pd.merge(three_months, regions, on="REGION")
merged_data = pd.merge(merged_data, six_months, on="REGION")
merged_data = pd.merge(merged_data, one_year, on="REGION")
merged_data = pd.merge(merged_data, three_years, on="REGION")
merged_data = pd.merge(merged_data, cron, on="REGION")
print merged_data

    REGION  status_3_m STATE  status_6_m  status_1_y  status_3_y  WaitingTime
0        1   99.698304    RI   99.596962   99.479312   99.283618   257.038528
1        1   99.698304    MA   99.596962   99.479312   99.283618   257.038528
2        1   99.698304    CT   99.596962   99.479312   99.283618   257.038528
3        1   99.698304    ME   99.596962   99.479312   99.283618   257.038528
4        1   99.698304    NH   99.596962   99.479312   99.283618   257.038528
5        1   99.698304    VT   99.596962   99.479312   99.283618   257.038528
6        2   98.995900    DE   98.631302   98.128087   97.169416   207.228608
7        2   98.995900    DC   98.631302   98.128087   97.169416   207.228608
8        2   98.995900    MD   98.631302   98.128087   97.169416   207.228608
9        2   98.995900    NJ   98.631302   98.128087   97.169416   207.228608
10       2   98.995900    PA   98.631302   98.128087   97.169416   207.228608
11       2   98.995900    WV   98.631302   98.128087   97.169416

In [11]:
merged_data = merged_data[["REGION", "STATE", "status_3_m", "status_6_m", "status_1_y", "status_3_y", "WaitingTime"]]
print merged_data

    REGION STATE  status_3_m  status_6_m  status_1_y  status_3_y  WaitingTime
0        1    RI   99.698304   99.596962   99.479312   99.283618   257.038528
1        1    MA   99.698304   99.596962   99.479312   99.283618   257.038528
2        1    CT   99.698304   99.596962   99.479312   99.283618   257.038528
3        1    ME   99.698304   99.596962   99.479312   99.283618   257.038528
4        1    NH   99.698304   99.596962   99.479312   99.283618   257.038528
5        1    VT   99.698304   99.596962   99.479312   99.283618   257.038528
6        2    DE   98.995900   98.631302   98.128087   97.169416   207.228608
7        2    DC   98.995900   98.631302   98.128087   97.169416   207.228608
8        2    MD   98.995900   98.631302   98.128087   97.169416   207.228608
9        2    NJ   98.995900   98.631302   98.128087   97.169416   207.228608
10       2    PA   98.995900   98.631302   98.128087   97.169416   207.228608
11       2    WV   98.995900   98.631302   98.128087   97.169416

In [12]:
merged_data.to_csv("../data/liver_status_new.csv", index=False)